<a href="https://colab.research.google.com/github/RogerioMatos75/colab_colmap/blob/main/colab/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
    <picture>
    <source media="(prefers-color-scheme: dark)" srcset="https://docs.nerf.studio/_images/logo-dark.png">
    <source media="(prefers-color-scheme: light)" srcset="https://docs.nerf.studio/_images/logo.png">
    <img alt="nerfstudio" src="https://docs.nerf.studio/_images/logo.png" width="400">
    </picture>
</p>


# Nerfstudio: A collaboration friendly studio for NeRFs


![GitHub stars](https://img.shields.io/github/stars/nerfstudio-project/nerfstudio?color=gold&style=social)

This colab shows how to train and view NeRFs from Nerfstudio both on pre-made datasets or from your own videos/images.

\\

Credit to [NeX](https://nex-mpi.github.io/) for Google Colab format.

## Frequently Asked Questions

*  **Downloading custom data is stalling (no output):**
    * This is a bug in Colab. The data is processing, but may take a while to complete. You will know processing completed if `data/nerfstudio/custom_data/transforms.json` exists. Terminating the cell early will result in not being able to train.
*  **Processing custom data is taking a long time:**
    * The time it takes to process data depends on the number of images and its resolution. If processing is taking too long, try lowering the resolution of your custom data.
*  **Error: Data processing did not complete:**
    * This means that the data processing script did not fully complete. This could be because there were not enough images, or that the images were of low quality. We recommend images with little to no motion blur and lots of visual overlap of the scene to increase the chances of successful processing.
*   **Training is not showing progress**:
    * The lack of output is a bug in Colab. You can see the training progress from the viewer.
* **Viewer Quality is bad / Low resolution**:
    * This may be because more GPU is being used on training that rendering the viewer. Try pausing training or decreasing training utilization.
* **WARNING: Running pip as the 'root' user...:**:
    * This and other pip warnings or errors can be safely ignored.
* **Other problems?**
    * Feel free to create an issue on our [GitHub repo](https://github.com/nerfstudio-project/nerfstudio).


In [ ]:
# @markdown <h1>Install Nerfstudio and Dependencies (~8 min)</h1>

%cd /content/
!pip install --upgrade pip

# Installing PyTorch with CUDA 11.8 support (compatible with Python 3.12)
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118

# Installing COLMAP
%cd /content/
!apt-get install colmap

# Install nerfstudio
%cd /content/
!pip install git+https://github.com/nerfstudio-project/nerfstudio.git

In [49]:
# @markdown <h1> Downloading and Processing Data</h1>
# @markdown <h3>Pick the preset scene or upload your own images/video</h3>
import glob
import os

from google.colab import files
from IPython.core.display import HTML, display

scene = "📤 upload your images"  # @param ['🖼 poster', '🚜 dozer', '🌄 desolation', '📤 upload your images' , '🎥 upload your own video', '🔺 upload Polycam data', '💽 upload your own Record3D data']
scene = " ".join(scene.split(" ")[1:])

if scene == "upload Polycam data":
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    if len(uploaded.keys()) > 1:
        print("ERROR, upload a single .zip file when processing Polycam data")
    dataset_dir = [os.path.join(dir, f) for f in uploaded.keys()][0]
    !ns-process-data polycam --data $dataset_dir --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene == "upload your own Record3D data":
    display(HTML("<h3>Zip your Record3D folder, and upload.</h3>"))
    display(
        HTML(
            '<h3>More information on Record3D can be found <a href="https://docs.nerf.studio/en/latest/quickstart/custom_dataset.html#record3d-capture" target="_blank">here</a>.</h3>'
        )
    )
    %cd /content/
    !mkdir -p /content/data/nerfstudio/custom_data
    %cd /content/data/nerfstudio/custom_data/
    uploaded = files.upload()
    dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    record_3d_zipfile = preupload_datasets[0]
    !unzip $record_3d_zipfile -d /content/data/nerfstudio/custom_data
    custom_data_directory = glob.glob("/content/data/nerfstudio/custom_data/*")[0]
    !ns-process-data record3d --data $custom_data_directory --output-dir /content/data/nerfstudio/custom_data/
    scene = "custom_data"
elif scene in ["upload your images", "upload your own video"]:
    display(HTML("<h3>Select your custom data</h3>"))
    display(HTML("<p/>You can select multiple images by pressing ctrl, cmd or shift and click.<p>"))
    display(
        HTML(
            "<p/>Note: This may take time, especially on higher resolution inputs, so we recommend to download dataset after creation.<p>"
        )
    )
    !mkdir -p /content/data/nerfstudio/custom_data
    if scene == "upload your images":
        !mkdir -p /content/data/nerfstudio/custom_data/raw_images
        %cd /content/data/nerfstudio/custom_data/raw_images
        uploaded = files.upload()
        dir = os.getcwd()
    else:
        %cd /content/data/nerfstudio/custom_data/
        uploaded = files.upload()
        dir = os.getcwd()
    preupload_datasets = [os.path.join(dir, f) for f in uploaded.keys()]
    del uploaded
    %cd /content/

    if scene == "upload your images":
        !ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/ --ImageReader.camera_model SIMPLE_RADIAL
    else:
        video_path = preupload_datasets[0]
        !ns-process-data video --data $video_path --output-dir /content/data/nerfstudio/custom_data/ --ImageReader.camera_model SIMPLE_RADIAL

    scene = "custom_data"
else:
    %cd /content/
    !ns-download-data nerfstudio --capture-name=$scene

print("Data Processing Succeeded!")

/content/data/nerfstudio/custom_data/raw_images


Saving frame_0001.png to frame_0001 (3).png
Saving frame_0002.png to frame_0002 (3).png
Saving frame_0003.png to frame_0003 (3).png
Saving frame_0004.png to frame_0004 (3).png
Saving frame_0005.png to frame_0005 (3).png
Saving frame_0006.png to frame_0006 (3).png
Saving frame_0007.png to frame_0007 (3).png
Saving frame_0008.png to frame_0008 (3).png
Saving frame_0009.png to frame_0009 (3).png
Saving frame_0010.png to frame_0010 (3).png
Saving frame_0011.png to frame_0011 (3).png
Saving frame_0012.png to frame_0012 (3).png
Saving frame_0013.png to frame_0013 (3).png
Saving frame_0014.png to frame_0014 (3).png
Saving frame_0015.png to frame_0015 (3).png
Saving frame_0016.png to frame_0016 (3).png
Saving frame_0017.png to frame_0017 (3).png
Saving frame_0018.png to frame_0018 (3).png
Saving frame_0019.png to frame_0019 (3).png
Saving frame_0020.png to frame_0020 (3).png
Saving frame_0021.png to frame_0021 (3).png
Saving frame_0022.png to frame_0022 (3).png
Saving frame_0023.png to frame_0

In [52]:
# @markdown <h1>Start Training</h1>

%cd /content
!pip install colab-xterm
%load_ext colabxterm
%env TERM=xterm
from IPython.display import clear_output

clear_output(wait=True)
if os.path.exists(f"data/nerfstudio/{scene}/transforms.json"):
    print(
        "\033[1m"
        + "Copy and paste the following command into the terminal window that pops up under this cell."
        + "\033[0m"
    )
    print(
        f"ns-train nerfacto --viewer.websocket-port 7007 --viewer.make-share-url True nerfstudio-data --data data/nerfstudio/{scene} --downscale-factor 4"
    )
    print()
    %xterm
else:
    from IPython.core.display import HTML, display

    display(HTML('<h3 style="color:red">Error: Data processing did not complete</h3>'))
    display(HTML("<h3>Please re-run `Downloading and Processing Data`, or view the FAQ for more info.</h3>"))

In [ ]:
# @title # Render Video { vertical-output: true }
# @markdown <h3>Export the camera path from within the viewer, then run this cell.</h3>
# @markdown <h5>The rendered video should be at renders/output.mp4!</h5>


base_dir = "/content/outputs/unnamed/nerfacto/"
training_run_dir = base_dir + os.listdir(base_dir)[0]

from IPython.core.display import HTML, display

display(HTML("<h3>Upload the camera path JSON.</h3>"))
%cd $training_run_dir
uploaded = files.upload()
uploaded_camera_path_filename = list(uploaded.keys())[0]

config_filename = training_run_dir + "/config.yml"
camera_path_filename = training_run_dir + "/" + uploaded_camera_path_filename
camera_path_filename = camera_path_filename.replace(" ", "\\ ").replace("(", "\\(").replace(")", "\\)")

%cd /content/
!ns-render camera-path --load-config $config_filename --camera-path-filename $camera_path_filename --output-path renders/output.mp4

In [50]:
!ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/

[20:47:42] 🎉 Done copying images with prefix 'frame_'.                                        ]8;id=376812;file:///usr/local/lib/python3.12/dist-packages/nerfstudio/process_data/process_data_utils.py\process_data_utils.py]8;;\:]8;id=850321;file:///usr/local/lib/python3.12/dist-packages/nerfstudio/process_data/process_data_utils.py#348\348]8;;\
(    ● ) Copying images...
──────────────────────────────────────────────  💀 💀 💀 ERROR 💀 💀 💀  ───────────────────────────────────────────────
Error running command: colmap feature_extractor --database_path /content/data/nerfstudio/custom_data/colmap/database.db 
--image_path /content/data/nerfstudio/custom_data/images --ImageReader.single_camera 1 --ImageReader.camera_model OPENCV
--SiftExtraction.use_gpu 1
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "" even th

In [51]:
!ns-process-data images --data /content/data/nerfstudio/custom_data/raw_images --output-dir /content/data/nerfstudio/custom_data/

[20:51:12] 🎉 Done copying images with prefix 'frame_'.                                        ]8;id=222022;file:///usr/local/lib/python3.12/dist-packages/nerfstudio/process_data/process_data_utils.py\process_data_utils.py]8;;\:]8;id=503960;file:///usr/local/lib/python3.12/dist-packages/nerfstudio/process_data/process_data_utils.py#348\348]8;;\
(    ● ) Copying images...
──────────────────────────────────────────────  💀 💀 💀 ERROR 💀 💀 💀  ───────────────────────────────────────────────
Error running command: colmap feature_extractor --database_path /content/data/nerfstudio/custom_data/colmap/database.db 
--image_path /content/data/nerfstudio/custom_data/images --ImageReader.single_camera 1 --ImageReader.camera_model OPENCV
--SiftExtraction.use_gpu 1
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
qt.qpa.xcb: could not connect to display 
qt.qpa.plugin: Could not load the Qt platform plugin "xcb" in "" even th